# LLMS

## Set up your `OPENAI_API_KEY`

In [ ]:
import os

if os.environ.get("OPENAI_API_KEY") is None:
   os.environ["OPENAI_API_KEY"] = "paste your api key here"

## Install Packages to Call the LLM

For this part we set up the LLM.

In [ ]:
!pip3 install openai

In [ ]:
import openai
import textwrap

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []

    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines

    return '\n'.join(wrapped_lines)


def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(prompt)
    print("")
    print("Response: ")
    print(response)
    print(80*"=")

def get_completion(prompt, temperature=0.0):
    # Create the OpenAI LLM
    client = openai.OpenAI()
    chat_completion = client.chat.completions.create(
                           model="gpt-3.5-turbo",
                           messages=[{"role": "user", "content": prompt}],
                           temperature=temperature,
                           max_tokens=1024)

    response = chat_completion.choices[0].message.content
    wrapped_response = wrap_text(response)
    return wrapped_response

## Calling an LLM

We can call the LLM by using the `get_completion` function. This takes
our prompt and delivers it to the LLM. The LLM returns a response. We
can print the response after we get it.

In [ ]:
#Set up a prompt
prompt = "Why is the sky blue? In 50 words or less."

#Call the LLM
response = get_completion(prompt)

#Print out the the response
print(response)

## Calling an LLM with the `temperature`

Temperature is one of the settings that most LLMs have. You can set the
temperature from 0.0 up to some maximum (like 1.0). Sometimes you can go
higher than 1.0.

Roughly

1.  Low temperature (eg 0.2-0.5)
    -   Generates more predictable and conservative text
2.  Medium temperature (eg. 0.6-0.8)
    -   Balances between creativity and coherence
3.  High temperature (eg. 0.9-1.2)
    -   Generates more creative diverse, and unpredictable text

### Calling with `temperature=0.0`

Here we call the LLM 3 times with temperature 0.0

In [ ]:
#Set up a prompt
prompt = "Why is the sky blue? In 50 words or less"

print("Calling the LLM with temperature 0")
for i in range(3):
  response = get_completion(prompt, temperature = 0.0)
  print(response)
  print("------")

### Calling with `temperature=0.4`

Here we call the LLM 3 times with temperature 0.4

In [ ]:
#Set up a prompt
prompt = "Why is the sky blue? In 50 words or less"

print("Calling the LLM with temperature 0.4")
for i in range(3):
  response = get_completion(prompt, temperature = 0.4)
  print(response)
  print("------")

### Calling with `temperature=1.0`

Here we call the LLM 3 times with temperature 1.0

In [ ]:
#Set up a prompt
prompt = "Why is the sky blue? In 50 words or less"

print("Calling the LLM with temperature 1.0")
for i in range(3):
  response = get_completion(prompt, temperature = 1.0)
  print(response)
  print("------")

## Calling the LLM using a template

Remember the template for having the LLM give us the name of a company
that makes a particular thing? Let’s call the LLM with that template and
template variable `Cars`:

In [ ]:
template = "Create a name for a company that makes {what}"

#Evaluate the template with what="Cars"
prompt=template.format(what="Cars")

#Call the LLM
response = get_completion(prompt)

#Print out the the response
print(response)

## Change the template variable `what`

Let’s call the LLM using `Books` as the template variable.

In [ ]:
#Evaluate the template with what="Books"
prompt=template.format(what="Books")
response = get_completion(prompt)

print(response)

## Add another template variable for `num`

In [ ]:
template = "Give {num} possible names for a company that makes {what}"

#Evaluate the template
prompt=template.format(num=3, what="Cars")
response = get_completion(prompt)

print(response)

### Formatting the response from the LLM

Lets see if we can introduce some formatting to the output:

In [ ]:
template ="""Give {num} possible names for a company that makes {what}. Give
your answer in {format}"""

#Evaluate the template
prompt=template.format(num=3, what="Cars", format="A bulleted list using upper case")
response = get_completion(prompt)

print(response)

## Calling the LLM multiple times with our template

The following is an example of calling the LLM multiple times, each time
with a different prompt that is created from our template.

In [ ]:
template ="""Give {num} possible names for a company that makes {what}. Give
your answer in {format}"""

myList = ["cars", "watches", "firecrackers"]

for item in myList:
  prompt = template.format(num=2, what=item, format="upper case")
  response = get_completion(prompt)
  print_prompt_and_response(prompt, response)